# Connection à la base de données 'fcd'

In [1]:
import pandas as pd
import pymongo

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

print('done')

done


# Lecture dans la base

In [2]:
cursor = collection.find()
entries = list(cursor)

#entries[:]

df = pd.DataFrame(entries)
df.head()
print('\ndone')


done


# Entrée 1 : géo

In [4]:
lon = float(input("longitude -> "))
lat = float(input("latitude -> "))
rad = float(input("rayon (km) -> "))

print('\ndone')

longitude -> -113.85
latitude -> 45.66
rayon (km) -> 15

done


# Entrée 2 : date

In [7]:
from datetime import timedelta
from datetime import datetime

year = int(input("Annee -> "))
month = int(input("Mois -> "))
day = int(input("Jour -> "))
hour = int(input("Heure -> "))
minutes = int(input("Minutes -> "))
seconds = int(input("Secondes -> "))
date = datetime(year, month, day, hour, minutes, seconds)

print('\ndone')

Annee -> 2020
Mois -> 3
Jour -> 5
Heure -> 14
Minutes -> 30
Secondes -> 11

done


# Entrée 3 : période

In [5]:
T = int(input("Période (minutes) -> "))
delta = timedelta(minutes = float(T/2))

datemin = date - delta
datemax = date + delta

datemin = datetime.timestamp(datemin)
datemax = datetime.timestamp(datemax)

print('\ndone')

Période (minutes) -> 46

done


# QUERY 1 

In [14]:
import pprint


# QUERY 1
# Calculer la densité [nombre de points] sur une Zone géographique Z et une période T 

def density_list(lon, lat, rad, datemin, datemax, option, zone, periode):

    dico={}
    query01={}
    query02={}
    center=[-1,-1]
    
    # points dans une zone Z
    if zone:
        
        center=[lon,lat]
        
        query01 = {
            "location.geo": {
                "$nearSphere": {
                    "$geometry": {
                        "type": "Point",
                        "coordinates": [ lon, lat]
                    },
                    "$minDistance": 0,
                    "$maxDistance": rad*1000 # mètres
                }
            }
        }

    # points à une période T
    if periode :
        query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}


    # requête finale
    query = {'$and': [query01, query02]}

    if option != {} :
        query = {'$and': [query, option]}
        
    liste=list(collection.find(query).distinct("asset_id"))
    dico['neighbors_id']=liste
    dico['density']=len(liste)
    dico['period']=[datemin, datemax]
    z = {}
    z['center'] = center
    z['radius'] = rad
    dico['zone'] = z
    
    return dico


collection = client.fcd.assets
#result = density_list(lon, lat, rad, datemin, datemax, {}, True, True)
#print ("density :", result['density'])

print('done')

done


# QUERY 2

In [15]:
import pprint

# QUERY 2
# Retourner le nombre de points voisins d'un asset X dans une période T.

def time_gap(current, new):

    td = abs(current - new)
    res = td/60
    return res


def asset_density(id_asset, rad, datemin, datemax):

    # les points d'un asset X
    query01 = {"asset_id": id_asset}

    # points à une période T
    query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}

    # les points d'un asset X à une période T
    query03 = {'$and': [query01, query02]}

    tmp = list(collection.find(query03))

    lon = 0
    lat = 0
    lon_average = 0
    lat_average = 0
    
    density = 0
    res = {}
    liste = []
    num = 0
    time = 0
    
    for doc in tmp:
        
        if time_gap(time, doc['recorded_at']) > 2:
            
            time = doc['recorded_at']
            
            lon = doc['location']['geo']['coordinates'][0]
            lat = doc['location']['geo']['coordinates'][1]         
            lon_average += lon
            lat_average += lat
            num += 1
            
            d_list = density_list(lon, lat, rad, datemin, datemax, {"asset_id": {"$ne": id_asset}}, True, True)

            if d_list['density'] > 0:

                for neighbor in d_list['neighbors_id']:

                    if neighbor not in liste:
                        density += 1
                        liste.append(neighbor)

    if density > 0:
        lon_average = lon_average/num
        lat_average = lat_average/num
        
        res['asset_id'] = id_asset
        res['density'] = density
        res['neighbors_id']= liste
        
        z = {}
        z['center'] = [lon_average, lat_average]
        z['radius'] = rad
        res['zone'] = z

    return res
  
    
collection = client.fcd.assets
'''result = asset_density('14429000019112', rad, datemin, datemax)

if result != {}:
    print('density :', result['density'])
    print('neighbors :',result['neighbors_id'])
'''
print('\ndone')


done


# QUERY 3

In [16]:
# QUERY 3
# Sur la base de tous les assets, retourner le Top – k [e.g. top 10] des zones les plus denses


def top_k_zones(k, rad, datemin, datemax):
    
    # list des id des différents asset
    ids_asset = list(collection.distinct("asset_id"))

    zones = []

    for id_ in ids_asset:

        zone = asset_density(id_, rad, datemin, datemax)

        if zone != {}:
            zones.append(zone)

    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


collection = client.fcd.assets
'''zones = top_k_zones(15, rad, datemin, datemax)

print("top", len(zones), "\n----------------------")
pprint.pprint(zones)
'''
print('\ndone')


done


# Query4

In [20]:
import math
import datetime

# Query 4
# Retournez le top k des périodes/ zones les plus denses 


# in meters
def distance(lat1, lat2, lon1, lon2):
    R = 6372800  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    d = 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return d


def top_k_zones2(k, rad):
    
    # toutes les points de la base de données
    base = list(collection.find())
    
    # tri selon la longitude puis la latitude
    base = sorted(base, key=lambda point:point['location']['geo']['coordinates'])
    print('sorting done')
    
    
    zones = []
    time = 0
    lon = 0
    lat = 0
    nextP = False
    
    for point in base:
        
        if distance(lat, point['location']['geo']['coordinates'][1],
                    lon, point['location']['geo']['coordinates'][0]) >= rad*1000*2:
            
            if len(zones) > 0:
                for z in zones:
                     
                    if distance(z['zone']['center'][1], point['location']['geo']['coordinates'][1],
                                z['zone']['center'][0], point['location']['geo']['coordinates'][0]) < rad*1000*2:
                        nextP = True
                        break
            if nextP:
                nextP = False
                continue
                
            lon = point['location']['geo']['coordinates'][0]
            lat = point['location']['geo']['coordinates'][1]
            zones.append(density_list(lon, lat, rad, 0, 0,{}, True, False))
            
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


def top_k_periods(k, delta, option):
    
    # toutes les points de la base de données
    base = list(collection.find())
    base = sorted(base, key=lambda point:point['recorded_at'])
    print('sorting done\n')
    
    
    zones = []
    time = 0
    
    # intervalle de temps en heures converti en secondes
    delta = datetime.timedelta(hours=delta).total_seconds()
    
    end = base[len(base)-1]['recorded_at']
    datemin = base[0]['recorded_at']
    datemax = datemin + delta
    
    
    while datemax < end:
        zones.append(density_list(-1, -1, -1, datemin, datemax, option, False, True))
             
        datemin = datemax
        datemax = datemin + delta
        
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


collection = client.fcd.assets

# top k des zones les plus denses
#zones = top_k_zones2(10, rad)

# top k des périodes les plus denses
zones = top_k_periods(5, 6, {})

pprint.pprint(zones)

print('\ndone')

sorting done

[{'density': 88,
  'neighbors_id': ['14256004530491',
                   '14256004535292',
                   '14256004569226',
                   '14256004574044',
                   '14256004592574',
                   '14256004664720',
                   '14295000006551',
                   '14429000003116',
                   '14429000004924',
                   '14429000005848',
                   '14429000006838',
                   '14429000008834',
                   '14429000008883',
                   '14429000008909',
                   '14429000010095',
                   '14429000012521',
                   '14429000014790',
                   '14429000015615',
                   '14429000015714',
                   '14429000016597',
                   '14429000019013',
                   '14429000019112',
                   '14429000022157',
                   '14429000023551',
                   '14429000026877',
                   '14429000029509',
       

# Query 5

In [22]:
# QUERY 5
# Retournez le top k des périodes T où nous avons le plus de voitures qui sont en circulation

# les trajets (sans stay points)
op = {"moving": 1}

zones = top_k_periods(5, 6, op)
pprint.pprint(zones)

print('\ndone')

sorting done

[{'density': 86,
  'neighbors_id': ['14256004535292',
                   '14256004559268',
                   '14256004644482',
                   '14256004664720',
                   '14295000006551',
                   '14295000017319',
                   '14429000003116',
                   '14429000004924',
                   '14429000005012',
                   '14429000005848',
                   '14429000006838',
                   '14429000008834',
                   '14429000008883',
                   '14429000008909',
                   '14429000010095',
                   '14429000012521',
                   '14429000014790',
                   '14429000015615',
                   '14429000015714',
                   '14429000016597',
                   '14429000019013',
                   '14429000019112',
                   '14429000022157',
                   '14429000023551',
                   '14429000026877',
                   '14429000029509',
       